# American Option Pricing via the Bellman Equation

An **American option** gives the holder the right to exercise at any time before maturity.
This is a classic **optimal stopping problem** governed by the Bellman equation:

$$V(S, t) = \max\left[ h(S), \, e^{-r\Delta t} \, \mathbb{E}[V(S', t+\Delta t)] \right]$$

where $h(S)$ is the intrinsic (exercise) value and the expectation is over the stock price dynamics.

We use the **Cox-Ross-Rubinstein (CRR) binomial model** to discretize the problem,
then solve it via backward induction — a finite-horizon Bellman equation.

In [ ]:
import numpy as np
import bellmaneq
from bellmaneq.viz import plot_exercise_boundary, plot_option_payoff, plot_price_convergence
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Pricing an At-the-Money American Put

Parameters:
- Spot price $S_0 = 100$
- Strike price $K = 100$
- Risk-free rate $r = 5\%$
- Volatility $\sigma = 20\%$
- Time to maturity $T = 1$ year

In [ ]:
spot = 100.0
strike = 100.0
rate = 0.05
vol = 0.20
maturity = 1.0
steps = 500

result = bellmaneq.price_american_option(
    spot=spot, strike=strike, rate=rate,
    volatility=vol, maturity=maturity, steps=steps
)

print(f'American Put Price: ${result.price:.4f}')
print(f'Steps: {steps}')
print(f'Exercise boundary points: {len(result.get_exercise_boundary())}')

## The Exercise Boundary

The **exercise boundary** $S^*(t)$ separates the exercise region from the continuation region:
- If $S < S^*(t)$: exercise immediately (for a put)
- If $S \geq S^*(t)$: continue holding

This boundary emerges naturally from the Bellman equation.

In [ ]:
time_steps = result.get_time_steps()
boundary = result.get_exercise_boundary()

fig = plot_exercise_boundary(
    time_steps, boundary,
    strike=strike,
    title='American Put Exercise Boundary (CRR Binomial Model)'
)
plt.show()

## Convergence with Number of Steps

As we increase the number of binomial steps, the discrete model converges
to the continuous Black-Scholes solution.

In [ ]:
steps_list = [10, 20, 50, 100, 200, 500, 1000]
prices = []

for n in steps_list:
    r = bellmaneq.price_american_option(
        spot=spot, strike=strike, rate=rate,
        volatility=vol, maturity=maturity, steps=n
    )
    prices.append(r.price)
    print(f'Steps={n:5d}: price=${r.price:.6f}')

fig = plot_price_convergence(
    steps_list, prices,
    title='American Put Price Convergence'
)
plt.show()

## Payoff Diagram

In [ ]:
stock_prices = result.get_stock_prices_at_maturity()

fig = plot_option_payoff(
    stock_prices, strike, result.price,
    is_call=False,
    title='American Put Payoff and Profit at Maturity'
)
plt.show()

## American Put vs Call

An American **call** on a non-dividend-paying stock should never be exercised early
(it equals the European call). An American **put**, however, has a genuine early exercise premium.

In [ ]:
put_result = bellmaneq.price_american_option(
    spot=spot, strike=strike, rate=rate,
    volatility=vol, maturity=maturity, steps=500
)
call_result = bellmaneq.price_american_option(
    spot=spot, strike=strike, rate=rate,
    volatility=vol, maturity=maturity, steps=500, is_call=True
)

print(f'American Put price:  ${put_result.price:.4f}')
print(f'American Call price: ${call_result.price:.4f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_exercise_boundary(
    put_result.get_time_steps(), put_result.get_exercise_boundary(),
    strike=strike, title='Put Exercise Boundary', ax=axes[0]
)
plot_exercise_boundary(
    call_result.get_time_steps(), call_result.get_exercise_boundary(),
    strike=strike, title='Call Exercise Boundary', ax=axes[1]
)

plt.tight_layout()
plt.show()

## Volatility Smile: Price Sensitivity

How does the American put price vary with volatility and moneyness?

In [ ]:
vols = np.linspace(0.1, 0.5, 9)
spots = np.linspace(80, 120, 9)

price_surface = np.zeros((len(vols), len(spots)))

for i, v in enumerate(vols):
    for j, s in enumerate(spots):
        r = bellmaneq.price_american_option(
            spot=s, strike=strike, rate=rate,
            volatility=v, maturity=maturity, steps=200
        )
        price_surface[i, j] = r.price

fig, ax = plt.subplots(figsize=(10, 7))
im = ax.imshow(
    price_surface, origin='lower', aspect='auto',
    extent=[spots[0], spots[-1], vols[0], vols[-1]],
    cmap='viridis'
)
ax.set_xlabel('Spot Price')
ax.set_ylabel('Volatility')
ax.set_title('American Put Price Surface')
plt.colorbar(im, label='Option Price ($)')

# Add contour lines
ax.contour(
    spots, vols, price_surface,
    levels=8, colors='white', linewidths=0.5, alpha=0.7
)
plt.show()